In [26]:
import pandas as pd
import numpy  as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.models import *
from keras.layers import *

In [3]:
train_data_path="/content/drive/MyDrive/train"

In [5]:
validation_data_path="/content/drive/MyDrive/validation"

In [6]:
training_datagen=ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [37]:
training_data=training_datagen.flow_from_directory(train_data_path, 
                                      target_size=(224,224), 
                                      batch_size=32,
                                      class_mode='categorical')

Found 1027 images belonging to 2 classes.


In [38]:
valid_datagen=ImageDataGenerator(rescale=1./255)

In [39]:
Validation_data=valid_datagen.flow_from_directory(validation_data_path, 
                                      target_size=(224,224), 
                                      batch_size=32,
                                      class_mode='categorical')

Found 256 images belonging to 2 classes.


In [40]:
model=VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [41]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [42]:
model.trainable=False

In [43]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPool2D()
])

In [44]:
model.layers

In [45]:
model.add(Dense(2,activation='softmax'))

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_max_pooling2d_1 (Glo  (None, 512)              0         
 balMaxPooling2D)                                                
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [47]:
model.compile(optimizer ='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model_path = '/content/drive/MyDrive/CNN_Horse_human_project/horse_or_human_predictor_VGG.h5'
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint] 

In [49]:
model.fit(training_data,epochs=10, verbose=1, validation_data= Validation_data,steps_per_epoch=1027//32,validation_steps=256//32,
              callbacks=callbacks_list)

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.8657 - accuracy: 0.4663
Epoch 1: val_accuracy improved from -inf to 0.63672, saving model to /content/drive/MyDrive/CNN_Horse_human_project/horse_or_human_predictor_VGG.h5
32/32 [==============================] - 27s 798ms/step - loss: 0.8657 - accuracy: 0.4663 - val_loss: 0.6151 - val_accuracy: 0.6367
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.5933 - accuracy: 0.6764
Epoch 2: val_accuracy improved from 0.63672 to 0.66797, saving model to /content/drive/MyDrive/CNN_Horse_human_project/horse_or_human_predictor_VGG.h5
32/32 [==============================] - 26s 812ms/step - loss: 0.5933 - accuracy: 0.6764 - val_loss: 0.5353 - val_accuracy: 0.6680
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.4324 - accuracy: 0.8261
Epoch 3: val_accuracy improved from 0.66797 to 0.85938, saving model to /content/drive/MyDrive/CNN_Horse_human_project/horse_or_human_predictor_VGG.h5
32/3

In [53]:
h1='/content/drive/MyDrive/test/validation/horses/horse1-000.png'
h2='/content/drive/MyDrive/test/validation/horses/horse1-122.png'
h3='/content/drive/MyDrive/test/validation/horses/horse1-224.png'

In [54]:
hu1='/content/drive/MyDrive/test/validation/humans/valhuman01-00.png'
hu2='/content/drive/MyDrive/test/validation/humans/valhuman01-03.png'
hu3='/content/drive/MyDrive/test/validation/humans/valhuman01-05.png'

In [55]:
def pred_human_horse(model, horse_or_human):
  test_image = image.load_img(horse_or_human, target_size = (224,224))
  test_image = image.img_to_array(test_image)/255
  test_image = np.expand_dims(test_image, axis = 0)
 
  result = model.predict(test_image).round(3)
 
  pred = np.argmax(result)
  print(result, "--->>>", pred)
 
  if pred == 0:
    print('Predicted>>> Horse')
  else:
    print('Predicted>>> Human')
 
"""## Predict Output"""
 
for horse_or_human in [h1,h2,h3, hu1,hu2,hu3]:
  pred_human_horse(model, horse_or_human)
 
# for horse_or_human in [h1,h2,h3,h4,h5,h6,h7,h8, hu1,hu2,hu3,hu4,hu5,hu6]:
#   pred_human_horse(model2, horse_or_human)
 
# for horse_or_human in [h1,h2,h3,h4,h5,h6,h7,h8, hu1,hu2,hu3,hu4,hu5,hu6]:
#   pred_human_horse(model3, horse_or_human)

[[0.967 0.033]] --->>> 0
Predicted>>> Horse
[[0.97 0.03]] --->>> 0
Predicted>>> Horse
[[0.889 0.111]] --->>> 0
Predicted>>> Horse
[[0.001 0.999]] --->>> 1
Predicted>>> Human
[[0.001 0.999]] --->>> 1
Predicted>>> Human
[[0.005 0.995]] --->>> 1
Predicted>>> Human


In [56]:
model.save('HorseVsHumanClassificationVGG16')

INFO:tensorflow:Assets written to: HorseVsHumanClassificationVGG16/assets
